In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import string
import pickle
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from collections import defaultdict
from wordcloud import WordCloud
from joblib import parallel_backend

import matplotlib.pyplot as plt

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Обучение модели на основе RandomForestClassifier


In [3]:
# Загрузка данных
data = pd.read_csv('/content/drive/MyDrive/cleaned_predicted_class_rus.csv')
# Загрузка выбросов
outliers = pd.read_csv('/content/drive/MyDrive/outliers_RUS.csv')

df_logic = pd.read_csv('/content/drive/MyDrive/df_topics_logic.csv')

In [4]:
# Функция для классификации текста
def classifactions(data):
  if data == 'Проблема с получением':
    return 'Проблема с получением'.lower()
  elif data == 'Соедините с оператором':
    return 'уточнить вопрос'.lower()
  elif data == 'Получатель подтверждает что заберт заказ':
    return 'уточнить вопрос'.lower()
  elif data == 'Срок доставки':
    return 'узнать статус посылки'.lower()
  elif data == 'Посылка не пришла':
    return 'Посылка не пришла'.lower()
  elif data == 'На кассе':
    return 'На кассе'.lower()
  elif data == 'Получить информацию о заказе':
    return 'узнать статус посылки'.lower()
  elif data == 'Отказаться от заказа':
    return 'уточнить вопрос'.lower()
  elif data == 'Проблема на пункте выдачи':
    return 'Проблема на пункте выдачи'.lower()
  elif data == 'Местонахождение заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать дату получения':
    return 'Узнать дату получения или адрес доставки'.lower()
  elif data == 'Заказ пришл':
    return 'Посылка пришла не на тот адрес или не дошла'.lower()
  elif data == 'Не работает постамат':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Проблемы с содержимым':
    return 'брак возврат или повреждение'.lower()
  elif data == 'Получить информацию о доставке':
    return 'узнать статус посылки'.lower()
  elif data == 'Узнать срок хранения':
    return 'Узнать срок хранения истек срок хранения'.lower()
  elif data == 'Заказ пришл не по тому адресу':
    return 'Заказ пришел но возникли проблемы с получением'.lower()
  elif data == 'Проблема с открытием ячейки':
    return 'не открывается ячейка или не работает постамат'.lower()
  elif data == 'Номер заказа':
    return 'узнать статус посылки'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Как забрать заказ':
    return 'Проблема с получением'.lower()
  elif data == 'Мне нужно взять посылку из постамата':
    return 'Проблема с получением'.lower()
  elif data == 'СМС для получения заказа':
    return 'уточнить вопрос'.lower()
  else:
    return 'уточнить вопрос'.lower()

data['topic'] = data.predicted_class.apply(lambda x: classifactions(x))

In [5]:
print(data.topic.value_counts())

topic
проблема с получением                             117750
уточнить вопрос                                    57310
узнать статус посылки                              27837
посылка не пришла                                  10576
на кассе                                            9360
проблема на пункте выдачи                           6851
не открывается ячейка или не работает постамат      5313
узнать дату получения или адрес доставки            4342
посылка пришла не на тот адрес или не дошла         3657
брак возврат или повреждение                        3020
узнать срок хранения истек срок хранения            2280
заказ пришел но возникли проблемы с получением      2136
Name: count, dtype: int64


In [6]:
# выбросам с большим количеством слов сразу присваиваем класс соеднить с оператором
outliers['topic'] = 'связать с оператором'

In [7]:
# объединяем логику, kmeans и выбросы
df_concatenated = pd.concat([df_logic[['text','topic']], data[['text','topic']]], ignore_index=True)
df_concatenated = pd.concat([df_concatenated[['text','topic']], outliers[['text','topic']]], ignore_index=True)
df_concatenated


,text,topic
0,потеряли код получения заказа,узнать код открытия ячейки
1,мне не пришел код для получения посылки,узнать код открытия ячейки
2,связаться с оператором,связать с оператором
3,я хочу узнать код получения посылки,узнать код открытия ячейки
4,оператора можно связаться,связать с оператором
...,...,...
1435236,остаюсь на линии мы бы хотели почему наш заказ...,связать с оператором
1435237,я заказ сделала заказ на горького пять мичурин...,связать с оператором
1435238,на кассе магазина пятерочка по улице энгельса ...,связать с оператором
1435239,а чтобы отключить то потом что надо будет сдел...,связать с оператором


In [8]:
df_concatenated.topic.value_counts(normalize=True)

topic
узнать код открытия ячейки                        0.272591
связать с оператором                              0.256087
узнать статус посылки                             0.184320
проблема с получением                             0.082042
не открывается ячейка или не работает постамат    0.080225
уточнить вопрос                                   0.052305
продлить срок хранения заказа                     0.017429
посылка не пришла                                 0.007369
вернуть заказ                                     0.006559
на кассе                                          0.006522
заказ потерян                                     0.005868
инвентаризация                                    0.005458
изменить адрес доставки заказа                    0.005394
проблема на пункте выдачи                         0.004773
узнать дату получения или адрес доставки          0.003025
посылка пришла не на тот адрес или не дошла       0.002548
изъятие посылки                                   

In [9]:
df_concatenated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435241 entries, 0 to 1435240
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1435241 non-null  object
 1   topic   1435241 non-null  object
dtypes: object(2)
memory usage: 21.9+ MB


In [10]:
df_concatenated.head(5)

,text,topic
0,потеряли код получения заказа,узнать код открытия ячейки
1,мне не пришел код для получения посылки,узнать код открытия ячейки
2,связаться с оператором,связать с оператором
3,я хочу узнать код получения посылки,узнать код открытия ячейки
4,оператора можно связаться,связать с оператором


In [11]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(df_concatenated['text'], df_concatenated['topic'], test_size=0.2, random_state=42)

In [12]:
# Преобразование текста в числовой формат
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Обучение модели
model = RandomForestClassifier()
model.fit(X_train_vec, y_train)

In [ ]:
# Оценка модели
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

In [ ]:
# Сохранение модели
# joblib.dump(model, 'model.pkl')